In [1]:
import numpy as np
import xarray as xr
import scipy
import gdal
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.filters import *
import os
import math


import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
%matplotlib inline

In [2]:
y = np.random.rand(25).reshape((5,5)).astype(np.float32)
practice_DEM = (y*10).astype(np.int)
practice_DEM

array([[8, 9, 2, 0, 4],
       [6, 3, 4, 7, 2],
       [0, 2, 9, 9, 1],
       [8, 7, 1, 6, 8],
       [9, 4, 5, 2, 3]])

In [3]:
practice_DEM[0,1]

9

In [4]:
def npCenteredSlope(y, dx):
    """Calculates slope by diferencing shifted vectors of length 2.
       y is a DEM read in as a numpy array."""
    dydx = (y[:,2:] - y[:,:-2])/(2*dx)
    return dydx

In [5]:
npCenteredSlope(practice_DEM,1)

array([[-3. , -4.5,  1. ],
       [-1. ,  2. , -1. ],
       [ 4.5,  3.5, -4. ],
       [-3.5, -0.5,  3.5],
       [-2. , -1. , -1. ]])

In [6]:
def assignBCs(elevGrid):
    
    """This function pads an input DEM by one cell along each edge. 
       The values assigned to the padded cells is taken from the 
       nearest value of the input DEM."""
    
    ny, nx = elevGrid.shape
    Zbc = np.zeros((ny + 2, nx + 2))
    Zbc[1:-1, 1:-1] = elevGrid
    
    #Assign boundary conditions - sides.
    Zbc[0, 1:-1] = elevGrid[0, :]
    Zbc[-1, 1:-1] = elevGrid[-1, :]
    Zbc[1:-1, 0] = elevGrid[:, 0]
    Zbc[1:-1, -1] = elevGrid[:, -1]
    
    #Assign boundary conditions - sides.
    Zbc[0, 0] = elevGrid[0, 0]
    Zbc[0, -1] = elevGrid[0, -1]
    Zbc[-1, 0] = elevGrid[-1, 0]
    Zbc[-1, -1] = elevGrid[-1, -1]
    
    return Zbc

In [7]:
assignBCs(practice_DEM)

array([[ 8.,  8.,  9.,  2.,  0.,  4.,  4.],
       [ 8.,  8.,  9.,  2.,  0.,  4.,  4.],
       [ 6.,  6.,  3.,  4.,  7.,  2.,  2.],
       [ 0.,  0.,  2.,  9.,  9.,  1.,  1.],
       [ 8.,  8.,  7.,  1.,  6.,  8.,  8.],
       [ 9.,  9.,  4.,  5.,  2.,  3.,  3.],
       [ 9.,  9.,  4.,  5.,  2.,  3.,  3.]])

In [15]:
def calcFiniteSlope(elevGrid, dx):
    
    """Calculates the slope given by a shifted vector of two cells first
       in the x direction and then y direction."""
    
    #Assign boundary conditions.
    Zbc = assignBCs(elevGrid)
    
    #Calculate dydx for shifted vector of size 2.
    Sx = (Zbc[1:-1, 2:] - Zbc[1:-1, :-2])/(2*dx)
    Sy = (Zbc[2:,1:-1] - Zbc[:-2, 1:-1])/(2*dx)
    
    slope_degrees = np.arctan(np.sqrt(Sx**2 + Sy**2)) * 57.29578
    
    return Sx, Sy, slope_degrees

In [16]:
calcFiniteSlope(practice_DEM, 1)

(array([[ 0.5, -3. , -4.5,  1. ,  2. ],
        [-1.5, -1. ,  2. , -1. , -2.5],
        [ 1. ,  4.5,  3.5, -4. , -4. ],
        [-0.5, -3.5, -0.5,  3.5,  1. ],
        [-2.5, -2. , -1. , -1. ,  0.5]]),
 array([[-1. , -3. ,  1. ,  3.5, -1. ],
        [-4. , -3.5,  3.5,  4.5, -1.5],
        [ 1. ,  2. , -1.5, -0.5,  3. ],
        [ 4.5,  1. , -2. , -3.5,  1. ],
        [ 0.5, -1.5,  2. , -2. , -2.5]]),
 array([[ 48.18968551,  76.73732464,  77.76044236,  74.63860524,
          65.90515801],
        [ 76.82528849,  74.63860524,  76.0679091 ,  77.76044236,
          71.06817742],
        [ 54.73561078,  78.52107795,  75.28564705,  76.0679091 ,
          78.69006819],
        [ 77.54542429,  74.63860524,  64.12331048,  78.57824701,
          54.73561078],
        [ 68.58328655,  68.19859109,  65.90515801,  65.90515801,
          68.58328655]]))

In [17]:
DEM_test = np.array([[ 50,  45,  50],
                      [30,  30,  30],
                      [8, 10, 10]],dtype = np.float32)

In [21]:
def slopeRate(DEM, x_cellsize, y_cellsize):
    
    a = DEM_test[0,0]; b = DEM_test[0,1]; c = DEM_test[0,2]
    d = DEM_test[1,0]; e = DEM_test[1,1]; f = DEM_test[1,2]
    g = DEM_test[2,0]; h = DEM_test[2,1]; i = DEM_test[2,2]
    
    dzdx = ((c + (2*f) + i) - (a + (2*d) + g)) / (8 * x_cellsize)
    dzdy = ((g + (2*h) + i) - (a + (2*b) + c)) / (8 * y_cellsize)
    
    rise_run = np.sqrt(dzdx**2 + dzdy**2)
    
    slope_degrees = np.arctan(rise_run) * (180/math.pi)
    
    return slope_degrees

In [22]:
slopeRate(DEM_test, 5, 5)

75.25765769167738